In [16]:
# Dependencies
import pandas as pd
import os
import numpy as np
import io

# load in School Data 
csv_path = os.path.join('raw_data', 'schools_complete.csv')
schools = pd.read_csv(csv_path)

# load in Student Data
csv_path = os.path.join('raw_data', 'students_complete.csv')
students = pd.read_csv(csv_path)

# Join Schools and Students (should be 1 to many)
school_student_pd = pd.merge(schools, students, how='inner', left_on="name", right_on="school",
         left_index=False, right_index=False, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate='1:m')
print("school_student_pd")
print(school_student_pd)

unique_schools = pd.Series(school_student_pd["School ID"]).unique()

total_schools = len(unique_schools)

unique_students = pd.Series(school_student_pd["Student ID"]).unique()
total_students = len(unique_students)


# Join Students and Schools (should be many to 1)   
student_school_pd = pd.merge(students, schools, how='inner', left_on="school", right_on="name",
         left_index=True, right_index=True, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate='m:1')
print("student_school_pd")
print(student_school_pd)
# Sum Budget
total_budget = student_school_pd["budget"].sum()

# Average Math Score
average_math_scores = students["math_score"].mean()

# Average Reading Score
average_reading_scores = students["reading_score"].mean()

# % Passing Math
passing_math = students.loc[students["math_score"] > 69].count()["Student ID"]
percent_passing_math = (passing_math/total_students) * 100

# % Passing Reading 
passing_reading = students.loc[students["reading_score"] > 69].count()["Student ID"]
percent_passing_reading = (passing_reading/total_students) * 100

# Overall Passing Rate: average of passing math and passing reading pcts
overall_passing_rate = (percent_passing_math + percent_passing_reading) /2

# Create a new table consolidating above calculations
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                 "Total Students": [total_students],
                                 "Total Budget": [total_budget],
                                 "Average Math Score": [average_math_scores],
                                 "Average Reading Score": [average_reading_scores],
                                 "% Passing Math":[percent_passing_math],
                                 "% Passing Reading":[percent_passing_reading],
                                 "% Overall Passing Rate": [overall_passing_rate],
                                   })

district_summary["Total Budget"] = district_summary["Total Budget"].map("${0:,.2f}".format)
col_list = ['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing Rate']
district_summary = district_summary[col_list]
  
writer = pd.ExcelWriter('district_summary.xlsx')
district_summary.to_excel(writer,'Sheet1')
district_summary

school_student_pd
       School ID              name_x      type  size   budget  Student ID  \
0              0   Huang High School  District  2917  1910635           0   
1              0   Huang High School  District  2917  1910635           1   
2              0   Huang High School  District  2917  1910635           2   
3              0   Huang High School  District  2917  1910635           3   
4              0   Huang High School  District  2917  1910635           4   
5              0   Huang High School  District  2917  1910635           5   
6              0   Huang High School  District  2917  1910635           6   
7              0   Huang High School  District  2917  1910635           7   
8              0   Huang High School  District  2917  1910635           8   
9              0   Huang High School  District  2917  1910635           9   
10             0   Huang High School  District  2917  1910635          10   
11             0   Huang High School  District  2917  1910

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [11]:
# For School Summary Begin with joined school to student dataframe 
school_student_pd = school_student_pd.rename(index=str, columns={"name_x": "school name", "name_y": "student name"})

# Insert Reading Score Passing Count Column: 1 = True and  0 = False (for % calculation later)
school_student_pd = school_student_pd.assign(rsp_count = (school_student_pd['reading_score']>69) - 0)

# Insert Math Score Passing Count Column: 1 = True and  0 = False (for % calculation later)
school_student_pd = school_student_pd.assign(msp_count = (school_student_pd['math_score']>69) - 0)
school_student_pd = school_student_pd.assign(combined_name = school_student_pd['school name'] + ":" + 
                                             school_student_pd['type'])
print(school_student_pd)
# Insert 

# Get School Counts of Students
school_counts = school_student_pd["school name"].value_counts()
print(school_counts)

# Establish groupby dataframe based on school name
grouped_by_school_name = school_student_pd.groupby(['school name'])

# School ID by School
school_id = grouped_by_school_name["School ID"].mean()

# Budget by School
total_school_budget = grouped_by_school_name["budget"].mean()

# Average Math Score by School
per_student_math_score = grouped_by_school_name["math_score"].mean()

# Average Reading Score by School
per_student_reading_score = grouped_by_school_name["reading_score"].mean()

# Passing Math Scores by School
passing_math_count = grouped_by_school_name["msp_count"].sum()

# Passing Reading Scores by School
passing_reading_count = grouped_by_school_name["rsp_count"].sum()

# Create New Summary Data Frame
schools_summary_table = pd.DataFrame({"Number of Students":school_counts,
                                      "Budget":total_school_budget,
                                     "Math Score":per_student_math_score,
                                     "Reading Score":per_student_reading_score,
                                     "Passing Math Count":passing_math_count,
                                     "Passing Reading Count":passing_reading_count,
                                     "School_ID":school_id})

# Insert Budget Spent Per Student column into Summary
schools_summary_table["Per Student Budget"] = schools_summary_table["Budget"] / schools_summary_table["Number of Students"]

# Insert % Passed Reading to Summary column into Summary
schools_summary_table["% Passing Math"] = (schools_summary_table["Passing Math Count"] / schools_summary_table["Number of Students"]) * 100

# Insert % Passed Reading to Summary column into Summary
schools_summary_table["% Passing Reading"] = (schools_summary_table["Passing Reading Count"] / schools_summary_table["Number of Students"]) * 100
schools_summary_table.head()

# Insert # Overall Passing Rate column into Summary
schools_summary_table["% Overall Passing Rate"] = (schools_summary_table["% Passing Reading"] + schools_summary_table["% Passing Math"]) / 2
print(schools_summary_table)
# Join with Schools df to pickup School Type and School Size  
schools_summary_table = pd.merge(schools_summary_table, schools, how='inner', left_on="School_ID", right_on="School ID",
                         left_index=False, right_index=False, sort=False,
                         suffixes=('_x', '_y'), copy=True, indicator=False,
                         validate='1:m')

schools_summary_table["type#"]  = ((schools_summary_table['type'] == 'District') + 1)

# Rename column names to more descri)ptive names
ss_table = schools_summary_table.rename(columns={
                                                'name': 'School Name',
                                                'type':'School Type',
                                                'Number of Students':'Total Students',
                                                'budget':'Total School Budget',
                                                'Math Score':'Average Math Score',
                                                'Reading Score':'Average Reading Score'
                                                })
# Select columns needed for the report
ss_table = ss_table[['School Name','School Type','Total Students','Total School Budget','Per Student Budget',
                      'Average Math Score','Average Reading Score','% Passing Math','% Passing Reading',
                    '% Overall Passing Rate']]

ss_table.set_index('School Name', inplace=True)

ss_table.index.name=''

ss_table["Total School Budget"] = ss_table["Total School Budget"].map("${0:,.2f}".format)
ss_table["Per Student Budget"] = ss_table["Per Student Budget"].map("${0:,.2f}".format)
writer = pd.ExcelWriter('school_summary.xlsx')
ss_table.to_excel(writer,'Sheet1')

# Show Final School Summary
ss_table


       School ID         school name      type  size   budget  Student ID  \
0              0   Huang High School  District  2917  1910635           0   
1              0   Huang High School  District  2917  1910635           1   
2              0   Huang High School  District  2917  1910635           2   
3              0   Huang High School  District  2917  1910635           3   
4              0   Huang High School  District  2917  1910635           4   
5              0   Huang High School  District  2917  1910635           5   
6              0   Huang High School  District  2917  1910635           6   
7              0   Huang High School  District  2917  1910635           7   
8              0   Huang High School  District  2917  1910635           8   
9              0   Huang High School  District  2917  1910635           9   
10             0   Huang High School  District  2917  1910635          10   
11             0   Huang High School  District  2917  1910635          11   

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


In [8]:
# Begin Top Performing Schools (By Passing Rate)
top_performing = ss_table.sort_values(['% Overall Passing Rate'],ascending=False)
writer = pd.ExcelWriter('top_performing.xlsx')
top_performing.head(5).to_excel(writer,'Sheet1')
top_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [9]:
bottom_performing = ss_table.sort_values(['% Overall Passing Rate'],ascending=True)
writer = pd.ExcelWriter('bottom_performing.xlsx')
bottom_performing.head(5).to_excel(writer,'Sheet1')
bottom_performing.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [10]:
# Math Scores by Grade 
# Select each grade and calculate the average for the math scores grouped by school
grade_9_scores = students[students['grade'] == '9th'].groupby('school')['math_score'].mean()
grade_10_scores = students[students['grade'] == '10th'].groupby('school')['math_score'].mean()
grade_11_scores = students[students['grade'] == '11th'].groupby('school')['math_score'].mean()
grade_12_scores = students[students['grade'] == '12th'].groupby('school')['math_score'].mean()

# Create New Summary Data Frame
math_scores_by_grade = pd.DataFrame({
                                "9th":grade_9_scores,
                                "10th":grade_10_scores,
                                "11th":grade_11_scores,
                                "12th":grade_12_scores
                                })
# Reorder Columns in df since we want 9th to be first column
math_scores_by_grade = math_scores_by_grade[['9th', '10th', '11th', '12th']]

math_scores_by_grade.index.name=''
writer = pd.ExcelWriter('math_scores_by_grade.xlsx')
math_scores_by_grade.to_excel(writer,'Sheet1')
top_performing.head(5)
math_scores_by_grade


,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [11]:
# Reading Scores by Grade 
# Select each grade and calculate the average for the math scores grouped by school
grade_9_scores = students[students['grade'] == '9th'].groupby('school')['reading_score'].mean()
grade_10_scores = students[students['grade'] == '10th'].groupby('school')['reading_score'].mean()
grade_11_scores = students[students['grade'] == '11th'].groupby('school')['reading_score'].mean()
grade_12_scores = students[students['grade'] == '12th'].groupby('school')['reading_score'].mean()

# Create New Summary Data Frame
reading_scores_by_grade = pd.DataFrame({
                                "9th":grade_9_scores,
                                "10th":grade_10_scores,
                                "11th":grade_11_scores,
                                "12th":grade_12_scores
                                })
# Reorder Columns in df since we want 9th to be first column
reading_scores_by_grade = reading_scores_by_grade[['9th', '10th', '11th', '12th']]

reading_scores_by_grade.index.name=''
writer = pd.ExcelWriter('reading_scores_by_grade.xlsx')
reading_scores_by_grade.to_excel(writer,'Sheet1')
reading_scores_by_grade

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [12]:
# Begin Spending Range Per Student Calculations

sp_range = schools_summary_table.rename(columns={'Math Score':'Average Math Score',
                                                  'Reading Score':'Average Reading Score'})
bins = [0, 580, 620, 640, 700] 
# Set up labels to compute when bins is updated
group_labels = ['<' + str(bins[1]-1), 
                str(bins[1]) + "-" + str(bins[2]-1), 
                str(bins[2]) + "-" + str(bins[3]-1),
                str(bins[3]) + "-" + str(bins[4])]
               
# Add new column with range information based values in Per Student Budget
sp_range['Spending Ranges (Per Student)'] = pd.cut(sp_range['Per Student Budget'],
                                                     bins, labels=group_labels)
# Get needed columns 
sp_range = sp_range[['Spending Ranges (Per Student)', 
                         'Average Math Score',
                         'Average Reading Score',
                         '% Passing Math',
                         '% Passing Reading',
                         '% Overall Passing Rate'
                        ]]
# Roll up to the Grouped By Object (Spending Ranges)
sp_range = sp_range.groupby("Spending Ranges (Per Student)")
writer = pd.ExcelWriter('spending_range_per_student.xlsx')
sp_range.max().to_excel(writer,'Sheet1')

# Show Final Spending Range Summary
sp_range.max()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<579,83.274201,83.989488,93.867718,96.539641,95.203679
580-619,83.839917,84.044699,94.594595,97.039828,95.586652
620-639,83.418349,83.848930,93.392371,97.308869,95.290520
640-700,77.289752,81.182722,68.309602,81.316421,73.807983


In [13]:
# Begin Size Range Calculations

sz_range = schools_summary_table.rename(columns={'Math Score':'Average Math Score',
                                                  'Reading Score':'Average Reading Score'
                                                  })
bins = [0, 1000, 2000, 5000] 
# Set up labels to compute when bins is updated
group_labels = ['Small(<' + str(bins[1]-1) + ')', 
                'Medium (' + str(bins[1]) + "-" + str(bins[2]-1) + ')', 
                'Large (' + str(bins[2]) + "-" + str(bins[3]) + ')']

# Add new column with range information based values in Per Student Budget
sz_range['School Size'] = pd.cut(sz_range['size'], bins, labels=group_labels)

# Get needed columns 
sz_range = sz_range[['School Size', 
                         'Average Math Score',
                         'Average Reading Score',
                         '% Passing Math',
                         '% Passing Reading',
                         '% Overall Passing Rate'
                        ]]
# Roll up to the Grouped By Object (School Size)
sz_range = sz_range.groupby('School Size')
writer = pd.ExcelWriter('spending_range_per_size.xlsx')
sz_range.max().to_excel(writer,'Sheet1')
# Show Final Spending Range Summary
sz_range.max()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<999),83.839917,84.044699,94.594595,96.252927,95.270270
Medium (1000-1999),83.682222,83.975780,94.133477,97.308869,95.586652
Large (2000-5000),83.274201,83.989488,93.867718,96.539641,95.203679


In [15]:
# Begin School Type Range Calculations

st_range = schools_summary_table.rename(columns={'Math Score':'Average Math Score',
                                                  'Reading Score':'Average Reading Score'
                                                  })
bins = [0, 1, 2] 
# Set up labels to compute when bins is updated
group_labels = [
                'Charter', 'District'
               ]

# Add new column with range information based values in Per Student Budget
st_range['School Type'] = pd.cut(st_range['type#'], bins, labels=group_labels)

# Get needed columns 
st_range = st_range[['School Type', 
                     'Average Math Score',
                     'Average Reading Score',
                     '% Passing Math',
                     '% Passing Reading',
                     '% Overall Passing Rate'
                    ]]
# Roll up to the Grouped By Object (School Type)
st_range = st_range.groupby('School Type')
writer = pd.ExcelWriter('school_type_range.xlsx')
st_range.max().to_excel(writer,'Sheet1')
# Show Final Spending Range Summary
st_range.max()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.839917,84.044699,94.594595,97.308869,95.586652
District,77.289752,81.182722,68.309602,81.933280,74.306672
